# 🏥 Análisis Geoespacial de Hospitales en Perú
## Notebook 4: Mapas Interactivos con Folium

### 🎯 Objetivos de este notebook:
- Crear mapas interactivos con Folium
- Mapa coroplético nacional con conteo de hospitales por distrito
- Cluster de marcadores para todos los hospitales
- Mapas de proximidad interactivos para Lima y Loreto
- Popups informativos y controles de capas
- Exportar mapas para el dashboard Streamlit

### 🌍 Mapas a crear:
1. **Mapa nacional**: Coroplético + cluster de hospitales
2. **Mapa Lima**: Proximidad con buffers interactivos
3. **Mapa Loreto**: Análisis de accesibilidad amazónica
4. **Mapas combinados**: Para integración en Streamlit

In [8]:
# 📚 IMPORTAR LIBRERÍAS
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster, HeatMap
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas para mapas interactivos")

✅ Librerías importadas para mapas interactivos


---
## 📥 1. Carga de Datos Procesados

Cargaremos todos los datos procesados de los notebooks anteriores para crear los mapas interactivos.

In [9]:
# 📥 CARGAR TODOS LOS DATOS PROCESADOS
def cargar_datos_interactivos():
    """Carga todos los datos necesarios para mapas interactivos"""
    
    print("📥 CARGANDO DATOS PARA MAPAS INTERACTIVOS...")
    print("=" * 45)
    
    datos = {}
    
    # Hospitales procesados
    try:
        datos['hospitales'] = gpd.read_file('../data/hospitales_procesados.geojson')
        print(f"✅ Hospitales: {len(datos['hospitales'])}")
    except FileNotFoundError:
        print("❌ Hospitales no encontrados")
        return None
    
    # Distritos con conteo de hospitales
    try:
        datos['distritos'] = gpd.read_file('../data/distritos_con_hospitales.geojson')
        print(f"✅ Distritos: {len(datos['distritos'])}")
    except FileNotFoundError:
        print("⚠️ Distritos con conteo no encontrados, usando shapefile base")
        datos['distritos'] = gpd.read_file('../data/DISTRITOS.shp').to_crs('EPSG:4326')
    
    # Buffers de proximidad
    for region in ['lima', 'loreto']:
        try:
            datos[f'buffers_{region}'] = gpd.read_file(f'../data/buffers_{region}_10km.geojson')
            datos[f'centros_{region}'] = gpd.read_file(f'../data/centros_poblados_{region}_acceso.geojson')
            print(f"✅ Datos proximidad {region.title()}")
        except FileNotFoundError:
            print(f"⚠️ Datos proximidad {region} no encontrados")
    
    return datos

# Cargar datos
datos_mapas = cargar_datos_interactivos()

if datos_mapas:
    print(f"\n📊 Resumen de datos cargados:")
    for key, value in datos_mapas.items():
        if value is not None:
            print(f"  • {key}: {len(value)} registros")

📥 CARGANDO DATOS PARA MAPAS INTERACTIVOS...
✅ Hospitales: 232
✅ Distritos: 1873
✅ Datos proximidad Lima
✅ Datos proximidad Loreto

📊 Resumen de datos cargados:
  • hospitales: 232 registros
  • distritos: 1873 registros
  • buffers_lima: 37 registros
  • centros_lima: 177 registros
  • buffers_loreto: 6 registros
  • centros_loreto: 18 registros


---
## 🗺️ 2. Mapa Nacional Interactivo

Crearemos el mapa nacional principal con coroplético de distritos y cluster de hospitales.

In [10]:
# 🗺️ CREAR MAPA NACIONAL INTERACTIVO
def crear_mapa_nacional(datos):
    """Crea mapa nacional con coroplético y cluster de hospitales"""
    
    print("🗺️ CREANDO MAPA NACIONAL INTERACTIVO...")
    print("=" * 40)
    
    # Calcular centro del mapa
    bounds = datos['hospitales'].total_bounds
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2
    
    # Crear mapa base
    mapa_nacional = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=6,
        tiles='OpenStreetMap'
    )
    
    # Añadir título
    title_html = '''
    <h3 align="center" style="font-size:20px"><b>🏥 Hospitales Públicos en Perú</b></h3>
    <p align="center">Análisis Geoespacial de Acceso Hospitalario</p>
    '''
    mapa_nacional.get_root().html.add_child(folium.Element(title_html))
    
    # Capa coroplética de distritos (si está disponible)
    if 'num_hospitales' in datos['distritos'].columns:
        print("📊 Añadiendo capa coroplética...")
        
        folium.Choropleth(
            geo_data=datos['distritos'].__geo_interface__,
            name='Hospitales por Distrito',
            data=datos['distritos'],
            columns=['IDDIST', 'num_hospitales'],
            key_on='feature.properties.IDDIST',
            fill_color='YlOrRd',
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='Número de Hospitales por Distrito',
            nan_fill_color='lightgray'
        ).add_to(mapa_nacional)
    else:
        # Solo mostrar límites de distritos
        print("📍 Añadiendo límites de distritos...")
        folium.GeoJson(
            datos['distritos'],
            style_function=lambda x: {
                'fillColor': 'lightgray',
                'color': 'white',
                'weight': 0.5,
                'fillOpacity': 0.3,
            },
            name='Límites Distritales'
        ).add_to(mapa_nacional)
    
    # Cluster de marcadores para hospitales
    print("🏥 Añadiendo cluster de hospitales...")
    marker_cluster = MarkerCluster(name='Hospitales').add_to(mapa_nacional)
    
    for idx, hospital in datos['hospitales'].iterrows():
        # Crear popup informativo
        popup_html = f"""
        <div style="width: 250px;">
            <h4 style="color: #E74C3C;"><b>{hospital['Nombre del establecimiento']}</b></h4>
            <hr>
            <p><b>📍 Ubicación:</b><br>
               {hospital['Departamento']}, {hospital.get('Provincia', 'N/A')}</p>
            <p><b>🏛️ Institución:</b> {hospital['Institución']}</p>
            <p><b>🏥 Clasificación:</b> {hospital.get('Clasificación', 'N/A')}</p>
            <p><b>📊 Coordenadas:</b><br>
               Lat: {hospital['latitud']:.4f}<br>
               Lon: {hospital['longitud']:.4f}</p>
        </div>
        """
        
        folium.Marker(
            location=[hospital['latitud'], hospital['longitud']],
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"{hospital['Nombre del establecimiento']}",
            icon=folium.Icon(color='red', icon='plus-sign', prefix='fa')
        ).add_to(marker_cluster)
    
    # Control de capas
    folium.LayerControl().add_to(mapa_nacional)
    
    print(f"✅ Mapa nacional creado con {len(datos['hospitales'])} hospitales")
    
    return mapa_nacional

# Crear mapa nacional
if datos_mapas:
    mapa_nacional = crear_mapa_nacional(datos_mapas)
    mapa_nacional

🗺️ CREANDO MAPA NACIONAL INTERACTIVO...
📊 Añadiendo capa coroplética...
🏥 Añadiendo cluster de hospitales...
✅ Mapa nacional creado con 232 hospitales


In [11]:
# 💾 GUARDAR Y MOSTRAR MAPA
def guardar_mapa_nacional(mapa):
    """Guarda el mapa y proporciona opciones para visualizarlo"""
    
    print("💾 GUARDANDO MAPA NACIONAL...")
    
    # Guardar como HTML
    ruta_mapa = '../data/mapa_nacional_hospitales.html'
    mapa.save(ruta_mapa)
    print(f"✅ Mapa guardado: {ruta_mapa}")
    
    # Obtener ruta absoluta
    import os
    ruta_absoluta = os.path.abspath(ruta_mapa)
    print(f"📂 Ruta completa: {ruta_absoluta}")
    
    print(f"\n🌐 Para ver el mapa:")
    print(f"1. Abre el archivo: {ruta_absoluta}")
    print(f"2. O navega a tu carpeta 'data' y abre 'mapa_nacional_hospitales.html'")
    print(f"3. Se abrirá en tu navegador por defecto")
    
    return ruta_absoluta

# Guardar mapa nacional
ruta_mapa_nacional = guardar_mapa_nacional(mapa_nacional)

# Mostrar información del mapa
print(f"\n📊 INFORMACIÓN DEL MAPA CREADO:")
print(f"✅ 232 hospitales con clusters interactivos")
print(f"✅ Coroplético por distritos")
print(f"✅ Popups informativos detallados")
print(f"✅ Control de capas")

💾 GUARDANDO MAPA NACIONAL...


✅ Mapa guardado: ../data/mapa_nacional_hospitales.html
📂 Ruta completa: c:\Users\bianq\Documents\Hospitals-Access-Peru\data\mapa_nacional_hospitales.html

🌐 Para ver el mapa:
1. Abre el archivo: c:\Users\bianq\Documents\Hospitals-Access-Peru\data\mapa_nacional_hospitales.html
2. O navega a tu carpeta 'data' y abre 'mapa_nacional_hospitales.html'
3. Se abrirá en tu navegador por defecto

📊 INFORMACIÓN DEL MAPA CREADO:
✅ 232 hospitales con clusters interactivos
✅ Coroplético por distritos
✅ Popups informativos detallados
✅ Control de capas


In [12]:
# 🗺️ MAPAS REGIONALES DE PROXIMIDAD
def crear_mapas_regionales_proximidad(datos):
    """Crear mapas de Lima y Loreto con buffers"""
    
    mapas_regionales = {}
    
    for region in ['lima', 'loreto']:
        if f'buffers_{region}' not in datos:
            print(f"⚠️ No hay datos de buffers para {region}")
            continue
            
        print(f"🗺️ Creando mapa {region.title()}...")
        
        # Datos de la región
        hospitales = datos['hospitales'][datos['hospitales']['Departamento'].str.upper() == region.upper()]
        buffers = datos[f'buffers_{region}']
        centros = datos[f'centros_{region}']
        
        # Centro del mapa
        center = hospitales.geometry.centroid.iloc[0]
        
        mapa = folium.Map(
            location=[center.y, center.x],
            zoom_start=9 if region == 'lima' else 7,
            tiles='CartoDB positron'
        )
        
        # Título del mapa
        title_html = f'''
        <h3 align="center"><b>Análisis de Proximidad - {region.title()}</b></h3>
        <p align="center">Buffers de 10 km y Acceso a Hospitales</p>
        '''
        mapa.get_root().html.add_child(folium.Element(title_html))
        
        # Buffers (círculos de 10 km)
        for idx, buffer in buffers.iterrows():
            folium.GeoJson(
                buffer.geometry,
                style_function=lambda x: {
                    'fillColor': 'lightblue',
                    'color': 'blue',
                    'weight': 2,
                    'fillOpacity': 0.2,
                    'opacity': 0.8
                },
                tooltip='Área de cobertura 10 km'
            ).add_to(mapa)
        
        # Hospitales
        for idx, hospital in hospitales.iterrows():
            folium.Marker(
                location=[hospital['latitud'], hospital['longitud']],
                popup=f"<b>{hospital['Nombre del establecimiento']}</b><br>{hospital['Institución']}",
                tooltip=hospital['Nombre del establecimiento'],
                icon=folium.Icon(color='red', icon='plus', prefix='fa')
            ).add_to(mapa)
        
        # Centros poblados
        for idx, centro in centros.iterrows():
            color = 'green' if centro['tiene_acceso'] else 'orange'
            icon_name = 'check' if centro['tiene_acceso'] else 'times'
            
            folium.CircleMarker(
                location=[centro['latitud'], centro['longitud']],
                radius=8,
                popup=f"""
                <b>{centro['nombre_ccpp']}</b><br>
                Acceso: {'✅ SÍ (≤10km)' if centro['tiene_acceso'] else '❌ NO (>10km)'}
                """,
                tooltip=centro['nombre_ccpp'],
                color=color,
                fill=True,
                fillColor=color,
                fillOpacity=0.8
            ).add_to(mapa)
        
        # Agregar leyenda
        legend_html = f'''
        <div style="position: fixed; 
                    bottom: 50px; left: 50px; width: 200px; height: 120px; 
                    background-color: white; border:2px solid grey; z-index:9999; 
                    font-size:14px; padding: 10px">
        <h4>Leyenda</h4>
        <p><i class="fa fa-plus" style="color:red"></i> Hospitales</p>
        <p><i class="fa fa-circle" style="color:green"></i> Con acceso (≤10km)</p>
        <p><i class="fa fa-circle" style="color:orange"></i> Sin acceso (>10km)</p>
        <p style="color:blue">⭕ Buffers 10km</p>
        </div>
        '''
        mapa.get_root().html.add_child(folium.Element(legend_html))
        
        # Guardar
        ruta = f'../data/mapa_{region}_proximidad.html'
        mapa.save(ruta)
        mapas_regionales[region] = ruta
        print(f"✅ Guardado: {ruta}")
        
        # Estadísticas
        acceso_stats = centros['tiene_acceso'].value_counts()
        print(f"   📊 {region.title()}: {acceso_stats.get(True, 0)} con acceso, {acceso_stats.get(False, 0)} sin acceso")
    
    return mapas_regionales

# Crear mapas regionales
mapas_regionales = crear_mapas_regionales_proximidad(datos_mapas)

🗺️ Creando mapa Lima...
✅ Guardado: ../data/mapa_lima_proximidad.html
   📊 Lima: 177 con acceso, 0 sin acceso
🗺️ Creando mapa Loreto...
✅ Guardado: ../data/mapa_loreto_proximidad.html
   📊 Loreto: 6 con acceso, 12 sin acceso


---
## 🌐 3. Mapa Comparativo Integrado

Crearemos un mapa que permita comparar ambas regiones lado a lado.

In [13]:
# 🌐 MAPA COMPARATIVO LIMA VS LORETO
def crear_mapa_comparativo():
    """Mapa con subdivisiones para comparar Lima vs Loreto"""
    
    print("🌐 CREANDO MAPA COMPARATIVO...")
    
    # Mapa base centrado en Perú
    mapa_comparativo = folium.Map(
        location=[-8.0, -76.0],
        zoom_start=6,
        tiles='CartoDB positron'
    )
    
    # Título
    title_html = '''
    <h2 align="center"><b>Comparación de Acceso Hospitalario: Lima vs Loreto</b></h2>
    <p align="center">Análisis de proximidad - Buffers de 10 km</p>
    '''
    mapa_comparativo.get_root().html.add_child(folium.Element(title_html))
    
    # Colores por región
    colors = {'LIMA': 'blue', 'LORETO': 'green'}
    
    # Añadir todos los hospitales con colores por región
    for region in ['LIMA', 'LORETO']:
        hospitales_region = datos_mapas['hospitales'][
            datos_mapas['hospitales']['Departamento'].str.upper() == region
        ]
        
        for idx, hospital in hospitales_region.iterrows():
            folium.Marker(
                location=[hospital['latitud'], hospital['longitud']],
                popup=f"""
                <b>{hospital['Nombre del establecimiento']}</b><br>
                📍 {hospital['Departamento']}<br>
                🏛️ {hospital['Institución']}
                """,
                tooltip=f"{region}: {hospital['Nombre del establecimiento']}",
                icon=folium.Icon(color=colors[region], icon='plus', prefix='fa')
            ).add_to(mapa_comparativo)
    
    # Añadir estadísticas como texto
    stats_html = '''
    <div style="position: fixed; 
                top: 100px; right: 50px; width: 300px; height: 200px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:12px; padding: 15px">
    <h4>📊 Estadísticas de Acceso</h4>
    <table style="width:100%">
        <tr><th>Región</th><th>Hospitales</th><th>Acceso</th></tr>
        <tr><td>Lima</td><td>37</td><td>100%</td></tr>
        <tr><td>Loreto</td><td>6</td><td>33.3%</td></tr>
    </table>
    <br>
    <p><b>Brecha:</b> 66.7 puntos porcentuales</p>
    <p style="font-size:10px; color:blue;">🔵 Lima | 🟢 Loreto</p>
    </div>
    '''
    mapa_comparativo.get_root().html.add_child(folium.Element(stats_html))
    
    # Guardar
    ruta_comparativo = '../data/mapa_comparativo_regiones.html'
    mapa_comparativo.save(ruta_comparativo)
    print(f"✅ Mapa comparativo guardado: {ruta_comparativo}")
    
    return ruta_comparativo

# Crear mapa comparativo
mapa_comparativo = crear_mapa_comparativo()

🌐 CREANDO MAPA COMPARATIVO...
✅ Mapa comparativo guardado: ../data/mapa_comparativo_regiones.html


In [14]:
# 📊 RESUMEN FINAL DE MAPAS INTERACTIVOS
def mostrar_resumen_mapas():
    """Muestra resumen de todos los mapas creados"""
    
    print("📊 RESUMEN FINAL - MAPAS INTERACTIVOS CREADOS")
    print("=" * 50)
    
    mapas_creados = [
        ("Mapa Nacional", "../data/mapa_nacional_hospitales.html", "Coroplético + 232 hospitales"),
        ("Mapa Lima Proximidad", "../data/mapa_lima_proximidad.html", "Buffers 10km + centros poblados"),
        ("Mapa Loreto Proximidad", "../data/mapa_loreto_proximidad.html", "Análisis amazónico disperso"),
        ("Mapa Comparativo", "../data/mapa_comparativo_regiones.html", "Lima vs Loreto lado a lado")
    ]
    
    for nombre, ruta, descripcion in mapas_creados:
        print(f"✅ {nombre}")
        print(f"   📂 {ruta}")
        print(f"   📝 {descripcion}")
        print()
    
    print("🎯 TODOS LOS MAPAS LISTOS PARA:")
    print("  • Integración en Streamlit")
    print("  • Visualización independiente")
    print("  • Análisis interactivo completo")

# Mostrar resumen
mostrar_resumen_mapas()

📊 RESUMEN FINAL - MAPAS INTERACTIVOS CREADOS
✅ Mapa Nacional
   📂 ../data/mapa_nacional_hospitales.html
   📝 Coroplético + 232 hospitales

✅ Mapa Lima Proximidad
   📂 ../data/mapa_lima_proximidad.html
   📝 Buffers 10km + centros poblados

✅ Mapa Loreto Proximidad
   📂 ../data/mapa_loreto_proximidad.html
   📝 Análisis amazónico disperso

✅ Mapa Comparativo
   📂 ../data/mapa_comparativo_regiones.html
   📝 Lima vs Loreto lado a lado

🎯 TODOS LOS MAPAS LISTOS PARA:
  • Integración en Streamlit
  • Visualización independiente
  • Análisis interactivo completo


---
## ✅ Notebook 4 Completado

### 🎯 Mapas Interactivos Finalizados

#### 🗺️ Mapas creados exitosamente:

1. **Mapa Nacional Interactivo**
   - ✅ Coroplético por distritos
   - ✅ Cluster de 232 hospitales
   - ✅ Popups informativos detallados

2. **Mapa Lima - Proximidad**
   - ✅ Buffers de 10 km interactivos
   - ✅ 37 hospitales con alta densidad
   - ✅ 100% de centros poblados con acceso

3. **Mapa Loreto - Proximidad**
   - ✅ Análisis de dispersión amazónica
   - ✅ 6 hospitales en región extensa
   - ✅ Identificación de centros críticos sin acceso

4. **Mapa Comparativo Regional**
   - ✅ Visualización Lima vs Loreto
   - ✅ Estadísticas de brecha de acceso
   - ✅ Contraste urbano-rural evidente

#### 🚀 Listo para el notebook final:
**05_streamlit_app.ipynb** - Dashboard interactivo completo

¡Análisis de mapas interactivos completado exitosamente!